## Import

In [10]:
import os
import cv2
from PIL import Image
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.optim.lr_scheduler import _LRScheduler

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from torchvision import models
from torchsummary import summary

# GPU 사용이 가능할 경우, GPU를 사용할 수 있게 함.'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
print(device)

print(os.environ.get('CUDA_VISIBLE_DEVICES'))

cuda
0


## Utils

In [11]:
# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# 클래스별 IoU를 계산하기 위한 함수
def calculate_iou_per_class(y_true, y_pred, class_id):
    intersection = np.sum((y_true == class_id) & (y_pred == class_id))
    union = np.sum((y_true == class_id) | (y_pred == class_id))
    iou = intersection / union if union > 0 else 0
    return iou

## Dataset, Data Loader

출력이미지 크기 키우기->ex) resnet 2048->1024->512->256 conv 256->512->1024->2048

In [12]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        directory_path = "/mnt/nas27/Dataset/Samsung_DM"
        img_path = self.data.iloc[idx, 1]
        img_path = os.path.join(directory_path, img_path[2:])
        image = cv2.imread(img_path)
        if image is None:
            raise ValueError(f"Failed to load image at {img_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image
        
        mask_path = self.data.iloc[idx, 2]
        mask_path = os.path.join(directory_path, mask_path[2:])
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            raise ValueError(f"Failed to load mask at {mask_path}")
        mask[mask == 255] = 12

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

    
    # 이미지 전처리 클래스
class ImageTransform():
  """
  훈련, 검증 동작 다르게 설정
  이미지 크기 resize, 색상 표준화
  훈련시 RandomResizedCrop, RandomHorizontalFilp으로 데이터 확장
  """
  def __init__(self, resize, mean, std):
    self.data_transform = {
        'train' : transforms.Compose([
            #transforms.RandomResizedCrop(
            #    resize, scale = (0.5, 1.0)), # 데이터 확장
            transforms.RandomHorizontalFlip(), # 데이터 확장
            transforms.ToTensor(), # Tensor로 변환
            transforms.Normalize(mean = mean, std = std) #표준화
        ]),
        'test': transforms.Compose([
            #transforms.Resize(resize), # Resize
            #transforms.CenterCrop(resize), # 중앙을 resize*resize로 crop
            transforms.ToTensor(), # Tensor로 변환
            transforms.Normalize(mean = mean, std = std) # 표준화
        ])

    }
  def __call__(self, img, phase = 'train'):
    """
    phase : 'train' or 'test'
    전처리 모드 지정
    """
    return self.data_transform[phase](img)


transform = A.Compose(
    [   
        A.Resize(224, 224),
        #A.Resize(128, 128),
        A.Normalize(),
        
        # 변형
        #A.VerticalFlip(p=0.5),
        # A.RandomRotate90(p=0.5),
        #A.HueSaturationValue(p=0.2),
        
        ToTensorV2()
    ]
)

In [13]:
print(CustomDataset.__len__)

<function CustomDataset.__len__ at 0x7f8e17ca2280>


Warmup

In [14]:
class WarmUpLR(_LRScheduler):
    def __init__(self, optimizer, total_iters, last_epoch=-1):
        self.total_iters = total_iters
        super(WarmUpLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        return [base_lr * self.last_epoch / self.total_iters for base_lr in self.base_lrs]


## Define Model

In [15]:
class HeadBlock(nn.Module):
    expansion = 4

    def __init__(self, in_channels,channels, stride=1):
        super(HeadBlock, self).__init__()

        # 1x1 convolution
        self.conv1 = nn.Conv2d(in_channels, channels, kernel_size=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(channels)
        self.relu1 = nn.ReLU(inplace=True)

        # 3x3 convolution
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(channels)
        self.relu2 = nn.ReLU(inplace=True)

        # 1x1 convolution
        self.conv3 = nn.Conv2d(channels, channels * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(channels * self.expansion)


        self.shortcut = nn.Sequential(
                nn.Conv2d(channels, channels * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(channels * self.expansion)
            )
        self.relu3 = nn.ReLU(inplace=True)

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu2(out)
         
        out = self.conv3(out)
        out = self.bn3(out)
        
        
        # Check the size of identity and out
        if identity.size() != out.size():
            identity = F.interpolate(identity, size=out.size()[2:])
        identity = self.shortcut(identity)
        out += identity
        out = self.relu3(out)
        
        return out


In [16]:
#Unet의 기본이 되는 conv블럭
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels,kernel_size = 3):
        super(ConvBlock, self).__init__()
        self.kernel_size = kernel_size
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=1)  # 여기서 in_channels는 out_channels와 동일해야 합니다.
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu2 = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        return x

class IdentityBlock(nn.Module):
    def __init__(self, in_channels, mid_channels, out_channels, stride=1):
        super(IdentityBlock, self).__init__()
        
        # 1x1 convolution
        self.conv1 = nn.Conv2d(in_channels, mid_channels, kernel_size=1, stride=stride, bias=False)
        self.bn1 = nn.BatchNorm2d(mid_channels)
        self.relu1 = nn.ReLU()

        # 3x3 convolution
        self.conv2 = nn.Conv2d(mid_channels, mid_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(mid_channels)
        self.relu2 = nn.ReLU()

        # 1x1 convolution
        self.conv3 = nn.Conv2d(mid_channels, out_channels, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.relu3 = nn.ReLU()
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu2(out)
         
        out = self.conv3(out)
        out = self.bn3(out)
        out = self.relu3(out)
        
        return out
class HeadBlock(IdentityBlock):
    def __init__(self, in_channels, mid_channels, out_channels, stride=1):
        super(HeadBlock, self).__init__(in_channels, mid_channels, out_channels, stride)
        
        self.shortcut = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(out_channels)
        )

    def forward(self, x):
        identity = x
        out = super().forward(x)
        
        if identity.size() != out.size():
            identity = F.interpolate(identity, size=out.size()[2:])
        identity = self.shortcut(identity)
        
        out += identity
        out = self.relu3(out)
        
        return out

#인코더 블럭
class Conv2(nn.Module):
    def __init__(self,in_channels, mid_channels, out_channels):
        super(Conv2,self).__init__() 
        self.headblock = HeadBlock(in_channels,mid_channels,out_channels)
        self.identityblock1 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.identityblock2 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
    def forward(self,x):
        x = self.headblock(x)
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        p = self.maxpool(x)
        return x , p
class Conv3(nn.Module):
    def __init__(self,in_channels, mid_channels, out_channels):
        super(Conv3,self).__init__() 
        self.headblock = HeadBlock(in_channels,mid_channels,out_channels)
        self.identityblock1 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.identityblock2 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.identityblock3 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
    def forward(self,x):
        x = self.headblock(x)
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        x = self.identityblock3(x)
        p = self.maxpool(x)
        return x , p
class Conv4(nn.Module):
    def __init__(self,in_channels, mid_channels, out_channels):
        super(Conv4,self).__init__() 
        self.headblock = HeadBlock(in_channels,mid_channels,out_channels)
        self.identityblock1 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.identityblock2 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.identityblock3 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.identityblock4 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.identityblock5 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
    def forward(self,x):
        x = self.headblock(x)
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        x = self.identityblock3(x)
        x = self.identityblock4(x)
        x = self.identityblock5(x)
        p = self.maxpool(x)
        return x , p
class Conv5(nn.Module):
    def __init__(self,in_channels, mid_channels, out_channels):
        super(Conv5,self).__init__() 
        self.headblock = HeadBlock(in_channels,mid_channels,out_channels)
        self.identityblock1 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.identityblock2 = IdentityBlock(out_channels,mid_channels,out_channels)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
    def forward(self,x):
        x = self.headblock(x)
        x = self.identityblock1(x)
        x = self.identityblock2(x)
        p = self.maxpool(x)
        return x , p
#디코더 블럭
class DecoderBlock(nn.Module):
    def __init__(self, channels):
        super(DecoderBlock, self).__init__()
        self.upsample = nn.ConvTranspose2d(channels*2, channels, kernel_size=4, stride=2, padding=1, output_padding=1) # output_padding 추가
        self.convblock1 = ConvBlock(channels*2, channels)

    def forward(self, x, skip):
        x = self.upsample(x)
        if x.size(2) != skip.size(2) or x.size(3) != skip.size(3):
            x = F.interpolate(x, size=(skip.size(2), skip.size(3)))
        x = torch.cat([x, skip], dim=1)
        x = self.convblock1(x)
        return x

#Unet구조 middle의 xm값의 움직임에 주의
class Unet(nn.Module):
    def __init__(self,n_classes):
        super(Unet,self).__init__()
        self.fconv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2)
        self.fbn1 = nn.BatchNorm2d(64)
        self.frelu1 = nn.ReLU()
        self.fconv2 = nn.Conv2d(64, 128, kernel_size=1, stride=1)
        self.fbn2 = nn.BatchNorm2d(128)
        self.frelu2 = nn.ReLU()
        self.fmaxpooling = nn.MaxPool2d(kernel_size=3,stride=2)
        
        self.conv2 = Conv2(128,64,256)
        self.conv3 = Conv3(256,128,512)
        self.conv4 = Conv4(512,256,1024)
        self.conv5 = Conv5(1024,512,2048)
        
        self.middleconv = ConvBlock(2048,4096)
        #self.dropout = nn.Dropout2d(0.2) #
           
        self.decoder5 = DecoderBlock(2048)
        self.decoder4 = DecoderBlock(1024)
        self.decoder3 = DecoderBlock(512)
        self.decoder2 = DecoderBlock(256)
        self.decoder1 = DecoderBlock(128)
        
        self.segmap = nn.Conv2d(128,n_classes, kernel_size=1)
        
        
    def forward(self,x):
        x = self.fconv1(x)#3->64
        x = self.fbn1(x)
        x = self.frelu1(x)
        x = self.fconv2(x)#3->64
        x = self.fbn2(x)
        x1 = self.frelu2(x)
        p = self.fmaxpooling(x)#첫 conv: x0([8, 64, 109, 109]) p([8, 64, 54, 54])
        #print("첫 conv:",x0.shape,p.shape)
        x2,p = self.conv2(p)#conv2:  x1:([8, 256, 54, 54]) p([8, 256, 26, 26])
        #print("conv2: ",x1.shape, p.shape)
        x3,p = self.conv3(p)#conv3:  x2([8, 512, 26, 26]) p([8, 512, 12, 12])
        #print("conv3: ",x2.shape, p.shape)
        x4,p = self.conv4(p)#conv4:  x3([8, 1024, 12, 12]) p([8, 1024, 5, 5])
        #print("conv4: ",x3.shape, p.shape)
        x5,p = self.conv5(p)#conv5:  x4([8, 2048, 5, 5]) p([8, 2048, 2, 2])
        #print("conv5: ",x4.shape, p.shape)
        xm = self.middleconv(p)#xm([8, 4096, 2, 2])
        #xm = self.dropout(xm)
        #print("mid:",xm.shape)
        x = self.decoder5(xm,x5)#뉴런:2048*2->2048 1
        #print(x.shape)
        x = self.decoder4(x,x4)#뉴런:1024*2->1024 
        #print(x.shape)
        x = self.decoder3(x,x3) #14
        #print(x.shape)
        x = self.decoder2(x,x2)#28
        #print(x.shape)
        x = self.decoder1(x,x1)#55
        
        x = F.interpolate(x, size=(224, 224))
        x = self.segmap(x)
        #print(x.shape)
        return x

## 파라미터 설정

In [17]:
BASE_LR = 0.01
END_LR = 0.01
EP = 30
BATCH_SIZE = 4
ACCMULATION_STEP = 3 
N_CLASSES = 13 #IoU 점수측정하기 위한 클래스의 개수
WUP_ITERS = 10  # 웜업을 위한 반복 횟수
# model 초기화
#model = Unet_resnet18(n_classes = N_CLASSES).to(device)
#model = ResNet50(num_classes=N_CLASSES).to(device)
model = Unet(n_classes = N_CLASSES).to(device)


# loss function과 optimizer 정의
criterion =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=BASE_LR)
optimizer.zero_grad() 
# Warmup을 위한 스케줄러 설정
scheduler_warmup = WarmUpLR(optimizer, WUP_ITERS)

dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./train_source.csv'), transform=transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8,pin_memory=True)
valid_dataset = CustomDataset(csv_file=os.path.join("/mnt/nas27/Dataset/Samsung_DM",'./val_source.csv'), transform=transform)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8,pin_memory=True)

## Model Train

In [18]:
import random
torch.cuda.empty_cache()
for epoch in range(EP):
    # 클래스별 IoU를 누적할 리스트 초기화
    train_class_ious = []
    # 학습
    model.train()
    epoch_loss = 0
    for images, masks in tqdm(dataloader):
        images = images.float().to(device)
        masks = masks.long().to(device)

        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        # if (epoch+1) % ACCMULATION_STEP == 0:
        #     optimizer.step()
        #     optimizer.zero_grad()
        #     # Warmup 스케줄러 업데이트
            # if epoch < WUP_ITERS:
            #     scheduler_warmup.step()

        epoch_loss += loss.item()
    '''
    #    # train 클래스별 IoU 계산
    #    outputs = torch.softmax(outputs, dim=1).cpu()
    #    outputs = torch.argmax(outputs, dim=1).numpy()

    #     for class_id in range(N_CLASSES):
    #         iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
    #         train_class_ious.append(iou)

    # train_class_ious = np.array(train_class_ious).reshape(N_CLASSES, -1)
    # train_class_ious = np.mean(train_class_ious, axis=1)
    # print("--IoU Scores Train--")
    # for class_id, iou in enumerate(train_class_ious):
    #     print(f'Class{class_id}: {iou:.4f}', end=" ")
    #     if (class_id+1) % 6 == 0:
    #         print()

    ## mIoU 계산
    #train_mIoU = np.mean(train_class_ious)
    '''
    # ___________________validation_____________________
    val_loss = 0
    val_class_ious = []  # 클래스별 IoU를 누적할 리스트 초기화
    with torch.no_grad():
        model.eval()
        for images, masks in tqdm(valid_dataloader):
            images = images.float().to(device)
            masks = masks.long().to(device)
            outputs = model(images)
            # validation loss 계산
            val_loss += criterion(outputs, masks).item()
            # validation 클래스별 IoU 계산
            outputs = torch.softmax(outputs, dim=1).cpu()
            outputs = torch.argmax(outputs, dim=1).numpy()
            for class_id in range(N_CLASSES):
                iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
                val_class_ious.append(iou)
    val_class_ious = np.array(val_class_ious).reshape(N_CLASSES, -1)
    val_class_ious = np.mean(val_class_ious, axis=1)
    print("--IoU Scores Valid--")
    for class_id, iou in enumerate(val_class_ious):
        print(f'Class{class_id}: {iou:.4f}', end=" ")
        if (class_id+1) % 6 == 0:
            print("")
    # mIoU 계산
    val_mIoU = np.mean(val_class_ious)
    # 에폭마다 결과 출력 
    print(f"\nEpoch{epoch+1}")
    print(f"Train Loss: {(epoch_loss/len(dataloader))}")
    print(f"Validation Loss: {val_loss/len(valid_dataloader)}, Validation mIoU Score: {val_mIoU:.4f}")
    print("___________________________________________________________________________________________\n")

  9%|▉         | 49/549 [00:19<03:20,  2.49it/s]


KeyboardInterrupt: 

## Inference

In [ ]:
# test_dataset = CustomDataset(csv_file='./test.csv', transform=transform, infer=True)
# test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

In [ ]:
# with torch.no_grad():
#     model.eval()
#     result = []
#     for images in tqdm(test_dataloader):
#         images = images.float().to(device)
#         outputs = model(images)
#         outputs = torch.softmax(outputs, dim=1).cpu()
#         outputs = torch.argmax(outputs, dim=1).numpy()
#         # batch에 존재하는 각 이미지에 대해서 반복
#         for pred in outputs:
#             pred = pred.astype(np.uint8)
#             pred = Image.fromarray(pred) # 이미지로 변환
#             pred = pred.resize((960, 540), Image.NEAREST) # 960 x 540 사이즈로 변환
#             pred = np.array(pred) # 다시 수치로 변환
#             # class 0 ~ 11에 해당하는 경우에 마스크 형성 / 12(배경)는 제외하고 진행
#             for class_id in range(12):
#                 class_mask = (pred == class_id).astype(np.uint8)
#                 if np.sum(class_mask) > 0: # 마스크가 존재하는 경우 encode
#                     mask_rle = rle_encode(class_mask)
#                     result.append(mask_rle)
#                 else: # 마스크가 존재하지 않는 경우 -1
#                     result.append(-1)
        

## Submission

In [ ]:
# submit = pd.read_csv('./sample_submission.csv')
# submit['mask_rle'] = result
# submit

In [ ]:
# submit.to_csv('./baseline_submit.csv', index=False)